<a href="https://colab.research.google.com/github/agalvezm/ACE2_scRNAseq/blob/master/notebooks_countmatrices/GSE117824_GSM3309835.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSE117824_GSM3309835

In [1]:
# define the values for the analysis

# accession id for the data
id = "GSE117824"
samp_id = "GSM3309835"

# If only bam available files, set bam = True, Fill link and filename
bam = True

if bam:
  # Assign link to python variable
  link_to_bam = "https://sra-pub-src-1.s3.amazonaws.com/SRR7613778/MF01.possorted_genome_bam.bam.1"
  
  # Assigns the link to the bash variable BAM_LINK. To be used by wget
  %env BAM_LINK=$link_to_bam

  # Assign filename to python variable. Used to convert to fastq and remove bam file to fastq after conversion

  bam_filename="MF01.possorted_genome_bam.bam.1"

if not bam:
  fastqs = ["",
            "",
            ]


env: BAM_LINK=https://sra-pub-src-1.s3.amazonaws.com/SRR7613778/MF01.possorted_genome_bam.bam.1


In [15]:
no_samples = 1

sample_id = [samp_id] * no_samples

database_id = [id] * no_samples

tissue = ["blood"] * no_samples

cell_type = ["PBMCs"] * no_samples

condition = ["diagnosis MF, age 53"] * no_samples


species = ["human"] * no_samples

technology = ["10xv2"] * no_samples

paper = ["Muus et al 2020"] * no_samples

figure = ["Fig 1 a,b  ED Fig 1 a,b,c,d  ED Fig 2 a,b,c,d,e"] * no_samples


# Set string variables for kb functions

species_kb = species[0]

technology_kb = technology[0]

# Imports and installs

In [3]:
# install and import necessary software

# Install kb and scanpy
!pip -q install kb-python 
!pip -q install scanpy

import re
import os

# Setup

import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import scanpy as sc
from scipy import stats

from collections import OrderedDict
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.preprocessing import scale

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from matplotlib import cm
from matplotlib.lines import Line2D

def nd(arr):
    return np.asarray(arr).reshape(-1)
def yex(ax):
    lims = [np.min([ax.get_xlim(), ax.get_ylim()]),
            np.max([ax.get_xlim(), ax.get_ylim()])]

    # now plot both limits against eachother
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    return ax

def trim_axs(axs, N):
    """little helper to massage the axs list to have correct length..."""
    axs = axs.flat
    for ax in axs[N:]:
        ax.remove()
    return axs[:N]

import warnings
warnings.filterwarnings('ignore')

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

     |████████████████████████████████| 59.1MB 79kB/s 
     |████████████████████████████████| 10.3MB 25.9MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 122kB 43.0MB/s 
     |████████████████████████████████| 13.2MB 51.2MB/s 
     |████████████████████████████████| 112kB 38.6MB/s 
     |████████████████████████████████| 51kB 2.6MB/s 
     |████████████████████████████████| 71kB 3.6MB/s 


# Downloads: (bam (if bam) and index

In [4]:
if bam:

  # Install bamtofastq from 10x website (only bam files available)
  !wget http://cf.10xgenomics.com/misc/bamtofastq-1.2.0
  !chmod +x bamtofastq-1.2.0
  # Download the bam file
  !wget -- continue ${BAM_LINK}




--2021-03-23 00:23:45--  http://cf.10xgenomics.com/misc/bamtofastq-1.2.0
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 104.18.1.173, 104.18.0.173, 2606:4700::6812:1ad, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.1.173|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cf.10xgenomics.com/misc/bamtofastq-1.2.0 [following]
--2021-03-23 00:23:45--  https://cf.10xgenomics.com/misc/bamtofastq-1.2.0
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.1.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13288280 (13M) [binary/octet-stream]
Saving to: ‘bamtofastq-1.2.0’

bamtofastq-1.2.0    100%[===================>]  12.67M  55.5MB/s    in 0.2s    

2021-03-23 00:23:46 (55.5 MB/s) - ‘bamtofastq-1.2.0’ saved [13288280/13288280]

--2021-03-23 00:23:46--  http://continue/
Resolving continue (continue)... failed: Name or service not known.
wget: unable to resolve host address ‘continue’
-

In [5]:
if bam:
  # Convert to fastq
  !./bamtofastq-1.2.0 --reads-per-fastq=500000000 $bam_filename ./fastqs\

  # Remove original bam file to save space
  !rm $bam_filename


bamtofastq v1.2.0
Args { arg_bam: "MF01.possorted_genome_bam.bam.1", arg_output_path: "./fastqs", flag_nthreads: 4, flag_locus: None, flag_bx_list: None, flag_reads_per_fastq: 500000000, flag_gemcode: false, flag_lr20: false, flag_cr11: false }
Writing finished.  Observed 254323137 read pairs. Wrote 254323137 read pairs


In [6]:
# Store fastq names on a list

if bam:
  # cd into fastqs folder
  %cd /content/fastqs

  #store the name of the folder generated by bamtofastq
  _filename = os.listdir()[0]

  # cd into that folder
  %cd $_filename

  # store fastq names in a list
  fastqs = os.listdir()


  # Remove I1 and R3 reads not relevant for our analysis

  # Initialize list containing elements to remove
  remov_elem = []

  print ("\n\nThis is the complete list of fastqs:\n -----------")
  for elem in fastqs:
    print (elem)

  # Search index (I1 or R3) fastqs and remove them from list
  for elem in fastqs:
    if re.search("_R3_", elem) or re.search("_I1_", elem):
      remov_elem = remov_elem +[elem]

  fastqs = [elem for elem in fastqs if elem not in remov_elem] 

  print ("\n\nThis is the filtered list of fastqs:\n -----------")
  for elem in fastqs:
    print (elem)


/content/fastqs
/content/fastqs/CALR042-TGAAGTAC_MissingLibrary_1_HYLYNBCXY


This is the complete list of fastqs:
 -----------
bamtofastq_S1_L002_R2_001.fastq.gz
bamtofastq_S1_L002_R1_001.fastq.gz
bamtofastq_S1_L001_R2_001.fastq.gz
bamtofastq_S1_L002_I1_001.fastq.gz
bamtofastq_S1_L001_R1_001.fastq.gz
bamtofastq_S1_L001_I1_001.fastq.gz


This is the filtered list of fastqs:
 -----------
bamtofastq_S1_L002_R2_001.fastq.gz
bamtofastq_S1_L002_R1_001.fastq.gz
bamtofastq_S1_L001_R2_001.fastq.gz
bamtofastq_S1_L001_R1_001.fastq.gz


In [7]:
# Remove fastqs that wont be analyzed to save space
if bam:
  for elem in remov_elem:
    !rm $elem

In [8]:
if bam:
  # sort fastqs alphabetically to get R1 and R2 in order
  fastqs = sorted(fastqs)

In [9]:
if bam:
  # Download the corresponding Kallisto index to folder containing fastqs
  !kb ref -d $species_kb -i index.idx -g t2g.txt -f1 transcriptome.fasta

if not bam:
  %cd /content

  # Download the corresponding Kallisto index to content folder
  !kb ref -d $species_kb -i index.idx -g t2g.txt -f1 transcriptome.fasta

[2021-03-23 01:18:39,341]    INFO Downloading files for human from https://caltech.box.com/shared/static/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz to tmp/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz
100% 2.23G/2.23G [01:39<00:00, 23.9MB/s]
[2021-03-23 01:20:20,681]    INFO Extracting files from tmp/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz


In [10]:
print(fastqs)

['bamtofastq_S1_L001_R1_001.fastq.gz', 'bamtofastq_S1_L001_R2_001.fastq.gz', 'bamtofastq_S1_L002_R1_001.fastq.gz', 'bamtofastq_S1_L002_R2_001.fastq.gz']


# Process fastq files (modify kb command according to fastqs list)


In [11]:
!cd

In [12]:
!ls

bamtofastq_S1_L001_R1_001.fastq.gz  bamtofastq_S1_L002_R2_001.fastq.gz
bamtofastq_S1_L001_R2_001.fastq.gz  index.idx
bamtofastq_S1_L002_R1_001.fastq.gz  t2g.txt


This code was manually changed from template to analyze all samples in all 4 folders

In [16]:
# Specify the sample number and whether they are paired-ended
number_of_samples = 1
paired_ended = True

if number_of_samples == 1:

  # Write the kb count command as a string with all fastqs of the list as an input
  cmd = "kb count --h5ad -i index.idx -g t2g.txt -x " + technology_kb + " -o output" + sample_id[0] + " "\
  + "--filter bustools -t 2 --overwrite "  +\
  "'/content/fastqs/CALR042-ATGCTCCG_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L001_R1_001.fastq.gz' " \
  "'/content/fastqs/CALR042-ATGCTCCG_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L001_R2_001.fastq.gz' " \
  "/content/fastqs/CALR042-ATGCTCCG_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L002_R1_001.fastq.gz " \
  "/content/fastqs/CALR042-ATGCTCCG_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L002_R2_001.fastq.gz " \
  "/content/fastqs/CALR042-CACTCGGA_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L001_R1_001.fastq.gz " \
  "/content/fastqs/CALR042-CACTCGGA_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L001_R2_001.fastq.gz " \
  "/content/fastqs/CALR042-CACTCGGA_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L002_R1_001.fastq.gz " \
  "/content/fastqs/CALR042-CACTCGGA_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L002_R2_001.fastq.gz " \
  "/content/fastqs/CALR042-GCTGAATT_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L001_R1_001.fastq.gz " \
  "/content/fastqs/CALR042-GCTGAATT_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L001_R2_001.fastq.gz " \
  "/content/fastqs/CALR042-GCTGAATT_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L002_R1_001.fastq.gz " \
  "/content/fastqs/CALR042-GCTGAATT_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L002_R2_001.fastq.gz " \
  "/content/fastqs/CALR042-TGAAGTAC_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L001_R1_001.fastq.gz " \
  "/content/fastqs/CALR042-TGAAGTAC_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L001_R2_001.fastq.gz " \
  "/content/fastqs/CALR042-TGAAGTAC_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L002_R1_001.fastq.gz " \
  "/content/fastqs/CALR042-TGAAGTAC_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L002_R2_001.fastq.gz"
  
  # Execute it
  !$cmd

# If more than one sample, iterate through fastqs accordingly
else:

  # Initializa counter for fastq files
  j = 0

  # Loop over samples for analysis
  for i in range(number_of_samples):

    # Write the kb count command as a string
    cmd = "kb count --h5ad -i index.idx -g t2g.txt -x " + technology_kb + " -o output" + sample_id[i] + " \
    --filter bustools -t 2 --overwrite " +\
    fastqs[j] + " " + fastqs[j+1]

    # Execute it
    !$cmd

    # Update j to move to the next fastq
    if paired_ended:
      j = j + 2
    else:
      j = j + 1




[2021-03-23 02:00:36,304]    INFO Using index index.idx to generate BUS file to outputGSM3309835 from
[2021-03-23 02:00:36,304]    INFO         /content/fastqs/CALR042-ATGCTCCG_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L001_R1_001.fastq.gz
[2021-03-23 02:00:36,304]    INFO         /content/fastqs/CALR042-ATGCTCCG_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L001_R2_001.fastq.gz
[2021-03-23 02:00:36,304]    INFO         /content/fastqs/CALR042-ATGCTCCG_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L002_R1_001.fastq.gz
[2021-03-23 02:00:36,304]    INFO         /content/fastqs/CALR042-ATGCTCCG_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L002_R2_001.fastq.gz
[2021-03-23 02:00:36,304]    INFO         /content/fastqs/CALR042-CACTCGGA_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L001_R1_001.fastq.gz
[2021-03-23 02:00:36,304]    INFO         /content/fastqs/CALR042-CACTCGGA_MissingLibrary_1_HYLYNBCXY/bamtofastq_S1_L001_R2_001.fastq.gz
[2021-03-23 02:00:36,304]    INFO         /content/fastqs/CALR042-CACTCGGA_M

In [ ]:
import time
time.sleep(6000000)

# Load unfiltered matrix and assign filters to each matrix individually

## Load the unfiltered matrix (check dimensions)

In [ ]:
# Define dict to store data
results = {}


In [ ]:
# load the unfiltered matrix
for i in range(number_of_samples):
  results[sample_id[i]] = anndata.read_h5ad("output" + sample_id[i] + "/counts_unfiltered/adata.h5ad")
  results[sample_id[i]].var["gene_id"] = results[sample_id[i]].var.index.values

  t2g = pd.read_csv("t2g.txt", header=None, names=["tid", "gene_id", "gene_name"], sep="\t")
  t2g.index = t2g.gene_id
  t2g = t2g.loc[~t2g.index.duplicated(keep='first')]

  results[sample_id[i]].var["gene_name"] = results[sample_id[i]].var.gene_id.map(t2g["gene_name"])
  results[sample_id[i]].var.index = results[sample_id[i]].var["gene_name"]
  print("The unfiltered matrix " + sample_id[i] + " contains {} cells by {} genes".format(len(results[sample_id[i]].obs), len(results[sample_id[i]].var)))

  results[sample_id[i]].obs["cell_counts"] = results[sample_id[i]].X.sum(axis=1)
  results[sample_id[i]].var["gene_counts"] = nd(results[sample_id[i]].X.sum(axis=0))

  results[sample_id[i]].obs["n_genes"] = nd((results[sample_id[i]].X>0).sum(axis=1))
  results[sample_id[i]].var["n_cells"] = nd((results[sample_id[i]].X>0).sum(axis=0))

  mito_genes = results[sample_id[i]].var_names.str.startswith("MT-" or "mt-") 
  results[sample_id[i]].obs["percent_mito"] = results[sample_id[i]][:,mito_genes].X.sum(axis=1)/results[sample_id[i]].X.sum(axis=1)*100

  # Changing the name of the index is necessary to write the file (it won't work with duplicated names)
  results[sample_id[i]].var.index.name = "index"



## Assign filters for each matrix individually

In [ ]:
# Modify this manually to change sample after having assigned the "expected_num_cells" and "mito_criteria" parameters
samp_n = 0


# Filtering criteria
cell_threshold = 100
gene_threshold = 3

mito_criteria = 30

In [ ]:

expected_num_cells = 10000#@param {type:"integer"}
knee = np.sort(nd(results[sample_id[i]].X.sum(axis=1)))[::-1]

fig, ax = plt.subplots(figsize=(5, 5))

x = knee
y = range(len(knee))

ax.loglog(x, y, linewidth=5, color="g")

ax.axvline(x=knee[expected_num_cells], linewidth=3, color="k")
ax.axhline(y=expected_num_cells, linewidth=3, color="k")

ax.set_xlabel("UMI Counts")
ax.set_ylabel("Set of Barcodes")

plt.show()

cell_threshold = knee[expected_num_cells]

results["cell_threshold" + sample_id[samp_n]] = knee[expected_num_cells]

print ("Cells were filtered down to " + str(expected_num_cells) + " with at least " + str(cell_threshold) + " UMIs")


mito_criteria = 18#@param {type:"integer"}
results["mito_criteria" + sample_id[samp_n]] = mito_criteria

fig, ax = plt.subplots(figsize=(5,5))


x = nd(results[sample_id[i]].obs["cell_counts"][results[sample_id[i]].obs["cell_counts"] > cell_threshold])
y = nd(results[sample_id[i]].obs["percent_mito"][results[sample_id[i]].obs["cell_counts"] > cell_threshold])

ax.scatter(x, y, color="green", alpha=0.1)

ax.axhline(y=mito_criteria, linestyle="--", color="k")


ax.set_xlabel("UMI Counts")
ax.set_ylabel("Percent mito")


plt.show()

print("We select " + str(mito_criteria) + " % as the mitochondrial content threshold")

# Filter matrix

In [ ]:
for i in range(number_of_samples):
  results[sample_id[i]].obs["pass_count_filter"] = results[sample_id[i]].obs["cell_counts"] > results["cell_threshold" + sample_id[i]]
  results[sample_id[i]].obs["pass_mito_filter"] = results[sample_id[i]].obs.percent_mito < results["mito_criteria" + sample_id[i]]
  results[sample_id[i]].var["pass_gene_filter"] = results[sample_id[i]].var["n_cells"] > gene_threshold

  cell_mask = np.logical_and(results[sample_id[i]].obs["pass_count_filter"].values, results[sample_id[i]].obs["pass_mito_filter"].values)
  gene_mask = results[sample_id[i]].var["pass_gene_filter"].values

  print("Current Shape: {:,} cells x {:,} genes".format(results[sample_id[i]].shape[0], results[sample_id[i]].shape[1]))
  print("    New shape: {:,} cells x {:,} genes".format(cell_mask.sum(), gene_mask.sum()))
  results["data_" + sample_id[i]] = results[sample_id[i]][cell_mask, gene_mask]

# Anotate and write the Anndata object

In [ ]:
for i in range(number_of_samples):


  results["data_" + sample_id[i]].uns["database_id"] = database_id[samp_n]

  results["data_" + sample_id[i]].uns["tissue"] = tissue[samp_n]

  results["data_" + sample_id[i]].uns["cell_type"] = cell_type[samp_n]

  results["data_" + sample_id[i]].uns["sample_id"] = sample_id[samp_n]

  results["data_" + sample_id[i]].uns["condition"] = condition[samp_n]

  results["data_" + sample_id[i]].uns["species"] = species[samp_n]

  results["data_" + sample_id[i]].uns["technology"] = technology[samp_n]

  results["data_" + sample_id[i]].uns["paper"] = paper[samp_n]

  results["data_" + sample_id[i]].uns["figure"] = figure[samp_n]

In [ ]:
%cd /content

for i in range(number_of_samples):

  results["data_" + sample_id[i]].write("result" + sample_id[i])